In [1]:
! pip install -U transformers
! pip install datasets sentence-transformers evaluate
! pip install accelerate -U

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/12/dd/f17b11a93a9ca27728e12512d167eb1281c151c4c6881d3ab59eb58f4127/transformers-4.35.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 40.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.0
    Uninstalling transformers-4.35.0:
      Successfully uninstalled transformers-4.35.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: fi

In [2]:
import numpy as np
import pandas as pd
import time
import re
from tqdm import notebook
import json

import seaborn as sns
import matplotlib.pyplot as plt

import os
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModel
from transformers import AutoConfig
from transformers import get_cosine_schedule_with_warmup

from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.metrics import r2_score

from datasets import Dataset
import evaluate

from sentence_transformers import SentenceTransformer, models, util

import gc

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, TextClassificationPipeline

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True

In [4]:
SEED = 97
seed_everything(SEED)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device: ', device.type)

Device:  cuda


In [6]:
def get_mnli_dataset_dfs(test_size=0.1):
    dataset = load_dataset("multi_nli")
    test_df = dataset["validation_mismatched"].to_pandas().dropna()
    test_df = test_df[['premise', 'hypothesis', 'label']]

    test_df.columns = ['sentence1', 'sentence2', 'label']

    test_df = test_df[test_df['label']!= -1].copy().reset_index(drop=True)

    return test_df

In [7]:
def get_snli_dataset_dfs(test_size=0.1):
    dataset = load_dataset("snli")

    # Access the train split of the dataset
    test_df = dataset["test"].to_pandas().dropna()

    test_df = test_df[['premise', 'hypothesis', 'label']]

    test_df.columns = ['sentence1', 'sentence2', 'label']

    test_df = test_df[test_df['label']!= -1].copy().reset_index(drop=True)

    return test_df

In [8]:
def get_anli_r1_dataset_dfs(test_size=0.1):
    dataset = load_dataset("anli")
    test_df = dataset["test_r1"].to_pandas().dropna()

    test_df = test_df[['premise', 'hypothesis', 'label']]
    test_df.columns = ['sentence1', 'sentence2', 'label']

    return test_df

In [9]:
def get_anli_r2_dataset_dfs(test_size=0.1):
    dataset = load_dataset("anli")
    test_df = dataset["test_r2"].to_pandas().dropna()

    test_df = test_df[['premise', 'hypothesis', 'label']]
    test_df.columns = ['sentence1', 'sentence2', 'label']

    return test_df

In [10]:
def get_anli_r3_dataset_dfs(test_size=0.1):
    dataset = load_dataset("anli")
    test_df = dataset["test_r3"].to_pandas().dropna()

    test_df = test_df[['premise', 'hypothesis', 'label']]
    test_df.columns = ['sentence1', 'sentence2', 'label']

    return test_df

In [11]:
def get_sick_dataset_dfs(test_size=0.1):
    dataset = load_dataset("sick")

    test_df = dataset["test"].to_pandas().dropna()

    test_df = test_df[['sentence_A', 'sentence_B', 'label']]

    test_df.columns = ['sentence1', 'sentence2', 'label']

    test_df = test_df[test_df['label']!= -1].copy().reset_index(drop=True)

    return test_df

In [12]:
def standardize_label(label):
    if label == 0:
        return 1
    elif label == 1:
        return 0
    return 2

def get_semeval_2014_taskl():
    dataset = load_dataset("sem_eval_2014_task_1")
    test_df = dataset["test"].to_pandas().dropna()

    test_df = test_df[['premise', 'hypothesis', 'entailment_judgment']]
    test_df['entailment_judgment'] = test_df['entailment_judgment'].apply(lambda x: standardize_label(x))
    test_df.columns = ['sentence1', 'sentence2', 'label']

    return test_df

In [13]:
test_dfs = {}

In [14]:
test_df = get_snli_dataset_dfs()
test_dfs['snli'] = test_df

test_df = get_mnli_dataset_dfs()
test_dfs['mnli'] = test_df

test_df = get_anli_r1_dataset_dfs()
test_dfs['anli_r1'] = test_df

test_df = get_anli_r2_dataset_dfs()
test_dfs['anli_r2'] = test_df

test_df = get_anli_r3_dataset_dfs()
test_dfs['anli_r3'] = test_df

test_df = get_sick_dataset_dfs()
test_dfs['sick'] = test_df

test_df = get_semeval_2014_taskl()
test_dfs['semeval'] = test_df

Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.9M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Dataset snli downloaded and prepared to /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b. Subsequent calls will reuse this data.


...

Generating train split:   0%|          | 0/4500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4927 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset sem_eval2014_task1 downloaded and prepared to /root/.cache/huggingface/datasets/sem_eval2014_task1/default/1.0.0/05e094e84ece42e036799e05c4d909ee68f6df8f82f60f9302b8ef15bb9de478. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
DATASET_NAME = "snli"

In [16]:
SAVED_METRICS_PATH = f"/kaggle/input/snli-run-metrics/run-metrics-0.json"
SAVED_MODELS_PATH = f"/kaggle/input/snli-models/SNLI_Models_BERT"

In [17]:
MODEL_NAME = "bert-base-uncased"
MAX_LEN = 128
# TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 128
# SAVE_MODEL_PATH = "/content/drive/MyDrive/Representative Subset/Models"
# SAVE_LOGS_PATH = "/content/drive/MyDrive/Representative Subset/Logs"
# LEARNING_RATE = 5e-5
# EPOCHS = 3

In [18]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [19]:
# Funtion to tokenize data
def tokenize_dataset(data):
    return tokenizer(data["sentence1"],
                     data["sentence2"],
                     max_length=MAX_LEN,
                     truncation=True,
                     padding="max_length"
                     )

In [20]:
def get_hf_dataset(df):
    data = Dataset.from_pandas(df)
    dataset = data.map(tokenize_dataset)
    return dataset

In [21]:
# Function to compute the metric
def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [22]:
with open(SAVED_METRICS_PATH, 'r') as f:
    data = json.load(f)

model_sizes = data['modelling_metrics']['subset_size']
model_sizes

[54935, 109870, 164805, 219740, 274675, 329610, 384545, 439480, 494415, 549350]

In [23]:
temp_training_args = TrainingArguments(
    output_dir='./results',
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    report_to='none'
)

In [24]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [25]:
# evaluation_metrics = []
# for size in model_sizes:
#     print(f"Loading model from: {SAVED_MODELS_PATH}/model-{size}")
#     model = AutoModelForSequenceClassification.from_pretrained(f"{SAVED_MODELS_PATH}/model-{size}").to(device)
#     trainer = Trainer(
#         model=model,                         # the instantiated 🤗 Transformers model to be trained
#         args=temp_training_args,                  # training arguments, defined above
#         compute_metrics=compute_metrics,     # the callback that computes metrics of interest
#     )

#     # Evaluate the model
#     eval_result = trainer.evaluate(eval_dataset=anli_r3_dataset)
#     print(eval_result)
#     evaluation_metrics.append(eval_result)

#     del model
#     gc.collect()

In [26]:
def save_metrics(filePath, evaluation_metrics, model_sizes):
    metrics = {'evaluation_metrics': evaluation_metrics, 'model_sizes': model_sizes}
    with open(filePath, "w") as outfile:
        json.dump(metrics, outfile)

In [27]:
for key, test_df in test_dfs.items():
    if key != DATASET_NAME:
        TEST_DATASET_NAME = key
        print(f"Testing for dataset: {TEST_DATASET_NAME} with {len(test_df)} samples.")
        test_dataset = get_hf_dataset(test_df)
        evaluation_metrics = []
        for size in model_sizes:
            print(f"Loading model from: {SAVED_MODELS_PATH}/model-{size}")
            model = AutoModelForSequenceClassification.from_pretrained(f"{SAVED_MODELS_PATH}/model-{size}").to(device)
            trainer = Trainer(
                model=model,                         # the instantiated 🤗 Transformers model to be trained
                args=temp_training_args,                  # training arguments, defined above
                compute_metrics=compute_metrics     # the callback that computes metrics of interest
            )

            # Evaluate the model
            eval_result = trainer.evaluate(eval_dataset=test_dataset)
            print(eval_result)
            evaluation_metrics.append(eval_result)

            del model
            gc.collect()
            
        save_metrics(f"/kaggle/working/eval-metrics-{TEST_DATASET_NAME}.json", evaluation_metrics, model_sizes)

Testing for dataset: mnli with 9832 samples.


/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


  0%|          | 0/9832 [00:00<?, ?ex/s]

Loading model from: /kaggle/input/snli-models/SNLI_Models_BERT/model-54935


{'eval_loss': 0.828673779964447, 'eval_accuracy': 0.6839910496338486, 'eval_runtime': 44.5633, 'eval_samples_per_second': 220.63, 'eval_steps_per_second': 1.728}
Loading model from: /kaggle/input/snli-models/SNLI_Models_BERT/model-109870


...

{'eval_loss': 1.6639348268508911, 'eval_accuracy': 0.5676882484270347, 'eval_runtime': 21.9633, 'eval_samples_per_second': 224.329, 'eval_steps_per_second': 1.776}
Loading model from: /kaggle/input/snli-models/SNLI_Models_BERT/model-384545


{'eval_loss': 1.8745654821395874, 'eval_accuracy': 0.5522630403896894, 'eval_runtime': 22.1253, 'eval_samples_per_second': 222.686, 'eval_steps_per_second': 1.763}
Loading model from: /kaggle/input/snli-models/SNLI_Models_BERT/model-439480


{'eval_loss': 1.6623032093048096, 'eval_accuracy': 0.5764156687639537, 'eval_runtime': 21.9815, 'eval_samples_per_second': 224.143, 'eval_steps_per_second': 1.774}
Loading model from: /kaggle/input/snli-models/SNLI_Models_BERT/model-494415


{'eval_loss': 1.6612308025360107, 'eval_accuracy': 0.5573371219809214, 'eval_runtime': 22.107, 'eval_samples_per_second': 222.871, 'eval_steps_per_second': 1.764}
Loading model from: /kaggle/input/snli-models/SNLI_Models_BERT/model-549350


{'eval_loss': 1.6315189599990845, 'eval_accuracy': 0.55774304850822, 'eval_runtime': 21.9273, 'eval_samples_per_second': 224.697, 'eval_steps_per_second': 1.779}
